In [64]:
import pickle
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
##nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('vader_lexicon')


def tf(term,doc):
    tokens = doc.lower().split()
    return tokens.count(term.lower())
    
def pullReviews(url):
    tempCritReviews=[]
    tempAudiReviews=[]
    for i in range(3):
        r = requests.get(url+'?page='+str(i+1)+'&sort=')
        soup = BeautifulSoup(r.text,'html5lib')
        divs = soup.findAll('div',{"class":"the_review"})
        for rev in divs:
            tempCritReviews.extend(sent_tokenize(rev.text)) ##3 pages of reviews, 20 reviews per page
    for i in range(3):
        r = requests.get(url+'?page='+str(i+1)+'&type=user')
        soup = BeautifulSoup(r.text,'html5lib')
        divs = soup.findAll('div',{"class":"user_review"})
        for rev in divs:
            tempAudiReviews.extend(sent_tokenize(rev.text)) 
    return tempCritReviews,tempAudiReviews

def vocabCheck(reviews):
    words = list()
    stopwords = set(nltk.corpus.stopwords.words('english'))
    stopwords.add("'s")
    stopwords.add("n't")
    for i in reviews:
        words.extend(map(lambda x: x.lower(),nltk.tokenize.word_tokenize(i)))
    for punctuation in ",.'?''`:;’”“`]""['``": #the last three are unicode characters of weird quotes. (python is unicode aware)
        stopwords.add(punctuation) #adds those individual punctuation to the stopwords to be filtered
    freqDist = nltk.FreqDist(words)
    sorted_terms = sorted(freqDist.items(), key=lambda x: x[1], reverse=True)
    tenMostCommon = [word for word in sorted_terms if word[0] not in stopwords][:10]
    print(tenMostCommon[9][0])

def sentimentCheck(reviews):
    analyzer = SentimentIntensityAnalyzer()
    avgPolarity=0
    positiveReviews=[]
    negativeReviews=[]
    for review in reviews:
        avgPolarity+=analyzer.polarity_scores(review)['compound']
        if(analyzer.polarity_scores(review)['pos']>analyzer.polarity_scores(review)['neg']):
            positiveReviews.append(review)
        elif(analyzer.polarity_scores(review)['pos']<analyzer.polarity_scores(review)['neg']):
            negativeReviews.append(review)
    avgPolarity=avgPolarity/len(reviews)
    print('pos list: ',len(positiveReviews))
    print('neg list: ',len(negativeReviews))
    print('avg pol: ',avgPolarity)
        
#criticReviews,audienceReviews = pullReviews('https://www.rottentomatoes.com/m/us_2019/reviews/')
#with open('criticReviews.pkl','wb') as f:
    #pickle.dump(criticReviews,f)
#with open('audienceReviews.pkl','wb') as f:
    #pickle.dump(audienceReviews,f)
with open('criticReviews.pkl','rb') as f:
    criticReviews = pickle.load(f)
with open('audienceReviews.pkl','rb') as f:
    audienceReviews = pickle.load(f)
#vocabCheck(criticReviews)
#vocabCheck(audienceReviews)
sentimentCheck(criticReviews)

pos list:  44
neg list:  28
avg pol:  0.09952068965517243
